In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib
matplotlib.use('TkAgg') #Nødvendige pakker
from collections import deque

#print(matplotlib.get_backend())

plt.rcParams['figure.figsize'] = [6, 6]
L = 5
x = np.linspace(-L,L,1000)
y = np.linspace(-L,L,1000)
X,Y = np.meshgrid(x,y)

In [4]:
def x_dot(x,y,mu,w):
    #return -x + w*y + x**2*y
    #return y
    #return mu-w*x
    return (mu-x**2-y**2)*x-w*y #normalformen for superkritiske hopf.
def y_dot(x,y,mu,w):
    #return mu - w*y - x**2*y
    #return mu*(1-x**2)*y-x
    #return 0.1*x-0.002*y
    #return w/(1+x**2)-y
    return(mu-x**2-y**2)*y+w*x
def solution_supercrit_Pol(t,mu,w,r0): #Polar coordinates
    C = (r0**2-mu)/(r0**2)
    T = 2*mu*t
    return (mu/(1-C*np.exp(-T)))**0.5

def solution_supercrit_Cart(t, mu,w, x0, y0): #Cartesian coordinates
    #theta = np.arctan2(y0,x0)
    return np.cos(w*t)*solution_supercrit_Pol(t, mu, w, x0), np.sin(w*t)*solution_supercrit_Pol(t, mu, w, y0) #x(t) and y(t)


def intgrator_rk4(xn, yn, dt, mu,w, D = 0):
    
    k1x = x_dot(xn, yn, mu,w)*dt
    k1y = y_dot(xn, yn, mu,w)*dt
    
    k2x = x_dot(xn+0.5*k1x, yn+0.5*k1y, mu,w)*dt
    k2y = y_dot(xn+0.5*k1x, yn+0.5*k1y, mu,w)*dt
    
    k3x = x_dot(xn+0.5*k2x, yn+0.5*k2y, mu,w)*dt
    k3y = y_dot(xn+0.5*k2x, yn+0.5*k2y, mu,w)*dt
    
    k4x = x_dot(xn+k3x,yn+k3y, mu,w)*dt
    k4y = y_dot(xn+k3x,yn+k3y, mu,w)*dt
    
    xnp = xn + 1./6*(k1x+2*k2x+2*k3x+k4x) + np.sqrt(2*D*dt)*np.random.normal(0,1)
    ynp = yn + 1./6*(k1y+2*k2y+2*k3y+k4y) + np.sqrt(2*D*dt)*np.random.normal(0,1)
    
    return xnp, ynp
def is_roughly_equal(x, list, tolerance):
    return any(np.isclose(x, list, atol = -0.001, rtol=tolerance))


In [11]:
mu = 0.5
w = 5
L = 10

x = np.linspace(-L,L,1000)
y = np.linspace(-L,L,1000)
X,Y = np.meshgrid(x,y)
#w = 5
#mu = 0.0006210526315789474
#mu = -3
plt.ion()
Tlist = np.linspace(0,3*np.pi,1000)
limitx, limity = solution_supercrit_Cart(Tlist,mu,1,np.sqrt(mu),np.sqrt(mu))
# Create the figure and axis
xdot = x_dot(X,Y,mu,w)
ydot = y_dot(X,Y,mu,w)
fig = plt.figure(figsize=(7,7))
ax  = plt.subplot(1,1,1)
#ax.plot(limitx,limity, label ="Limit cycle without noise")
plt.autoscale(enable=True, axis='both', tight=True)

P = 50
### Plot vector field (one vector every 40 points)
magnitude = np.sqrt(xdot**2 + ydot**2)
xdot_normalized = xdot / magnitude
ydot_normalized = ydot / magnitude
ax.quiver(X[::P, ::P], Y[::P, ::P], xdot_normalized[::P, ::P], ydot_normalized[::P, ::P],
         pivot='mid', units='inches')
#ax.plot(0,0, 'ro', label = "Fixpoint")

def poincare_slice(x1,x2,y):
    current_sign = np.sign(x1)  # Initialize with the sign of the first element in `x`
    prev_sign = np.sign(x2)
    return current_sign != prev_sign and y > 0
limitx,limity = [],[] 
dt = 0.01
running = True
def on_click(event):
    global running
    global limitx, limity
   #trajectory_label = ax.text(0, 0, 'Trajectory', color='red', fontsize=10, ha='center', va='center', alpha=0.7)

    if event.inaxes:    
        xmap, ymap = event.xdata, event.ydata
        sol_x, sol_y = deque(maxlen=int(2*np.pi/dt)), deque(maxlen=int(2*np.pi/dt))  # Use deque for efficient popping
        plotcolor = 'black'
        plt.title(f"Hopf bifurcation af Jacob  - Initial condition: ({xmap:.2f}, {ymap:.2f}), $\mu$ = {mu}, $\omega$ = {w}")
        # Initialize the plot line
        plot_line, = ax.plot([], [], ls='-', color=plotcolor, label = "Trajectory")
        plt.autoscale(enable=True, axis='both', tight=True)
        t = 0
        update_interval = 10  # Update plot every 10 iterations
        plt.legend()

        while running:
            t+=1
            sol_x.append(xmap)
            sol_y.append(ymap)
            xmap, ymap = intgrator_rk4(xmap, ymap, dt, mu, w)

            # Update plot every few iterations
            if t % update_interval == 0:#if len(sol_x) % update_interval == 0:
                plot_line.set_data(sol_x, sol_y)
                #trajectory_label.set_position((sol_x[-1], sol_y[-1]))
                
                plt.pause(0.001)

def on_close(event):
    global running
    running = False
    print("Close event triggered.")
    fig.canvas.mpl_disconnect(cid)  # Disconnect the click event
#plt.legend(loc = 'upper right')
# Connect the click event to the handler
cid = fig.canvas.mpl_connect('button_press_event', on_click)
cid_close = fig.canvas.mpl_connect('close_event', on_close)


Close event triggered.
